# (Attempted) Solving Sudoku with Linear Algebra

### Pretty-print matrix function

In [1]:
from IPython.display import HTML

def print_matrix(mat):
    t = ["<table>"]
    for row in mat:
        t.append("<tr>")
        for col in row:
            t.append("<td>" + str(col) + "</td>")
        t.append("</tr>")
    t.append("</table>")
    return HTML(''.join(t))

print_matrix([[1,2],[3,4]])

1,2
3,4


In [2]:
import numpy as np
import random

## 4x4 Sudoku

<table class="invisible-table"><tr><td>

Puzzle
<table class="padded-table">
<tr><td><b>1</b></td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
<tr><td> </td><td>&nbsp;</td><td><b>4</b></td><td>&nbsp;</td></tr>
<tr><td> </td><td><b>3</b></td><td>&nbsp;</td><td>&nbsp;</td></tr>
<tr><td>&nbsp;</td><td>&nbsp;</td><td><b>2</b></td><td>&nbsp;</td></tr>
</table>

</td><td>

Solution
<table class="padded-table">
<tr><td>1</td><td>4</td><td>3</td><td>2</td></tr>
<tr><td>3</td><td>2</td><td>4</td><td>1</td></tr>
<tr><td>2</td><td>3</td><td>1</td><td>4</td></tr>
<tr><td>4</td><td>1</td><td>2</td><td>3</td></tr>
</table>

</td></tr></table>

<style>
.padded-table td { padding: 15px; }
.invisible-table { border: 0px; }
</style>


In [3]:
m1 = [['a01', 'a02', 'a03', 'a04'],
      ['a05', 'a06', 'a07', 'a08'],
      ['a09', 'a10', 'a11', 'a12'],
      ['a13', 'a14', 'a15', 'a16']]

print_matrix(m1)

a01,a02,a03,a04
a05,a06,a07,a08
a09,a10,a11,a12
a13,a14,a15,a16


### System of Equations to describe puzzle

#### Sudoku-constant equations

These first 12 equations are the same for all 4x4 sudoku puzzles - they are the rules of the game:

By Rows:

* a01 + a02 + a03 + a04 = 10
* a05 + a06 + a07 + a08 = 10
* a09 + a10 + a11 + a12 = 10
* a13 + a14 + a15 + a16 = 10

By Columns:

* a01 + a05 + a09 + a13 = 10
* a02 + a06 + a10 + a14 = 10
* a03 + a07 + a11 + a15 = 10
* a04 + a08 + a12 + a16 = 10

By Quandrants:

* a01 + a02 + a05 + a06 = 10
* a03 + a04 + a07 + a08 = 10
* a09 + a10 + a13 + a14 = 10
* a11 + a12 + a15 + a16 = 10


#### Puzzle-specific equations

These last 4 equations are what we are given for this particular puzzle:

* a01 = 1
* a07 = 4
* a10 = 3
* a15 = 2



In [4]:
a =   [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

b = [10, 10, 10, 10,  10, 10, 10, 10,  10, 10, 10, 10,  1, 4, 3, 2]

In [5]:
x = np.linalg.lstsq(a,b)
np.reshape(x[0], (4,-1))

array([[ 1.        ,  3.47387651,  0.66316228,  4.86296121],
       [ 3.92477232,  1.60135117,  4.        ,  0.47387651],
       [ 2.04450137,  3.        ,  3.33683772,  1.61866091],
       [ 3.03072631,  1.92477232,  2.        ,  3.04450137]])

In [6]:
solution = np.reshape([int(round(i)) for i in x[0]], (4, -1))
solution

array([[1, 3, 1, 5],
       [4, 2, 4, 0],
       [2, 3, 3, 2],
       [3, 2, 2, 3]])

# Linear Regression Implementation

I wanted to implement my own linear regression algorithm to better understand how it works.

In [9]:
# TODO: Add precesion early break
def linear_regression(a, b, precision=0.01, learning_rate=0.1, max_iters = 200):
    m = len(b)
    omega = np.ones(m)
    
    for iter in range(max_iters):
        for i in range(m):
            omega[i] = omega[i] - learning_rate * (1.0/m) * sum((np.dot(a, omega) - b) * a[i])
        
    return omega

### Test problem to solve

In [10]:
# 3x+4y=7 and 5x+6y=8
a1=[[3,4],[5,6]]
b1=[7,8]
linear_regression(a1,b1)

array([-5. ,  5.5])

### Compare with numpy implementation's solution

In [11]:
((x,y),_,_,_) = np.linalg.lstsq(a1,b1)
(x,y)

(-4.9999999999999769, 5.4999999999999813)

In [12]:
result = linear_regression(a,b)
result

array([  4.71143849,   4.688242  ,   4.66519049,  -0.71713789,
         1.9568043 ,  -6.86351593,   1.06255254,  17.08008666,
        -1.34069241,  10.54590296,  -3.6327123 ,   7.43473406,
        -0.87012213,   7.11479335,  -4.74101031,  -2.80275689])

# Trying on Normal (9x9) Sudoku Puzzle

In [14]:
s9x9_flat = np.array(['a'+str(i) for i in range(81)])
s9x9 = np.reshape(s9x9_flat, (9, -1))

print_matrix(s9x9)

a0,a1,a2,a3,a4,a5,a6,a7,a8
a9,a10,a11,a12,a13,a14,a15,a16,a17
a18,a19,a20,a21,a22,a23,a24,a25,a26
a27,a28,a29,a30,a31,a32,a33,a34,a35
a36,a37,a38,a39,a40,a41,a42,a43,a44
a45,a46,a47,a48,a49,a50,a51,a52,a53
a54,a55,a56,a57,a58,a59,a60,a61,a62
a63,a64,a65,a66,a67,a68,a69,a70,a71
a72,a73,a74,a75,a76,a77,a78,a79,a80


### Generate equations (in matrix form)

In [19]:
sudoku_base = []
num_rows = 9
num_cols = 9

# Generate equations for rows
for row in range(num_rows):
    row_offset = num_rows * row
    new_row = [0 for i in range(num_rows * num_cols)]
    for i in range(num_rows):
        new_row[row_offset + i] = 1
    sudoku_base.append(new_row)

# Generate equations for columns
for col in range(num_cols):
    col_offset = num_cols * col
    new_row = [0 for i in range(num_rows * num_cols)]
    for i in range(num_cols):
        new_row[i * 9 + col] = 1
    sudoku_base.append(new_row)

# Generate equations for squares
current_cell = 0
for square_start in [0, 3, 6, 27, 30, 33, 54, 57, 60]:
    new_row = [0 for i in range(num_rows * num_cols)]
    for cell_offset in [0, 1, 2, 9, 10, 11, 18, 19, 20]:
        new_row[square_start+cell_offset] = 1
    sudoku_base.append(new_row)

In [20]:
# This is a wide matrix
#print_matrix(sudoku_base)

In [21]:
# Definition of sample puzzle
puzzle1 = {0:8, 3:1, 4:6, 9:4, 10:1, 
          12:9, 14:8, 17:6, 25:1, 31:8, 
          32:6, 35:3, 37:8, 38:9, 40:3, 
          42:2, 43:5, 45:3, 48:2, 49:9, 
          55:2, 63:7, 66:8, 68:4, 70:6, 
          71:5, 76:5, 77:7, 80:4}

In [24]:
def print_puzzle(puzzle):
    puzzle_mat = [' ' for i in range(81)]
    for (k,v) in puzzle.items():
        puzzle_mat[k] = v
    return print_matrix(np.reshape(puzzle_mat, (9, -1)))

In [25]:
print_puzzle(puzzle1)

8,,,1,6,,,,
4,1,,9,,8,,,6
,,,,,,,1,
,,,,8,6,,,3
,8,9,,3,,2,5,
3,,,2,9,,,,
,2,,,,,,,
7,,,8,,4,,6,5
,,,,5,7,,,4


In [28]:
print("Number of equations:", len(puzzle1)+len(a))

Number of equations: 45


In [29]:
print("Number of unknowns:", 81 - len(puzzle1))

Number of unknowns: 52


In [34]:
def get_puzzle_rep(puzzle):
    a = sudoku_base[:]
    b = [sum(range(1,10)) for i in range(len(a))]
    for (k,v) in puzzle.items():
        new_a_row = [0 for i in range(81)]
        new_a_row[k] = 1
        a.append(new_a_row)
        b.append(v)
    return (a,b)


In [35]:
p1_a, p1_b = get_puzzle_rep(puzzle1)

In [36]:
len(p1_a)

56

In [37]:
x = np.linalg.lstsq(p1_a,p1_b)
solution = np.reshape([int(round(i)) for i in x[0]], (9, -1))
solution

array([[  8,  75, -29,   1,   6,  -1, -10,  11, -15],
       [  4,   1,  11,   9,   3,   8,   3,   0,   6],
       [  6, -29,  -1,  17,   2,   0,  22,   1,  29],
       [ -8,   1,  11,   6,   8,   6,  10,   9,   3],
       [  8,   8,   9,  -6,   3,  11,   2,   5,   5],
       [  3,   1,  14,   2,   9,   5,   1,   6,   4],
       [  7,   2,  14,   6,   2,   5,  -1,   7,   4],
       [  7, -11,  17,   8,   7,   4,   2,   6,   5],
       [ 12,  -2,  -1,   2,   5,   7,  16,   1,   4]])

# Constrained Linear Regression - scipy.optimize.nnls

In [38]:
import scipy.optimize

In [39]:
result = scipy.optimize.nnls(p1_a, p1_b)

In [40]:
solution = np.reshape([int(round(i)) for i in result[0]], (9, -1))
solution

array([[ 8, 30,  0,  1,  6,  0,  0,  0,  0],
       [ 4,  1,  0,  9,  0,  8, 17,  0,  6],
       [ 0,  2,  0,  0,  1, 20, 21,  1,  0],
       [23,  0,  0,  0,  8,  6,  5,  0,  3],
       [ 0,  8,  9, 17,  3,  0,  2,  5,  1],
       [ 3,  2,  0,  2,  9,  0,  0,  3, 26],
       [ 0,  2, 21,  8, 13,  0,  0,  1,  0],
       [ 7,  0, 15,  8,  0,  4,  0,  6,  5],
       [ 0,  0,  0,  0,  5,  7,  0, 29,  4]])

# scipy.optimize.fmin_slsqp


In [41]:
import scipy.optimize

In [42]:
def objective_func(x):
    mat_result = sum((np.dot(p1_a, x) - p1_b)**2)
    #duplicate_count = 2 * sum([ (9 - len(set(x[row_start:row_start+9]))) for row_start in xrange(0, 81, 9)])
    return mat_result # + duplicate_count

In [43]:
objective_func(result[0])

4.833449373902292e-27

In [46]:
random_guess = [random.randint(1, 9) for i in range(81)]
objective_func(random_guess)

3291

In [50]:
slsqp_result = scipy.optimize.fmin_slsqp(func=objective_func, x0=random_guess, bounds=[(1,9) for i in range(81)])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.16510794728e-07
            Iterations: 34
            Function evaluations: 2852
            Gradient evaluations: 34


In [51]:
slsqp_result

array([ 7.99992986,  5.8155213 ,  5.55581589,  1.        ,  6.00007236,
        3.07285887,  5.70257946,  4.29829144,  5.55496279,  4.00010669,
        1.        ,  3.45899053,  9.        ,  4.49252577,  7.99996356,
        4.28944157,  4.75896454,  6.00000983,  7.35692941,  6.21084895,
        3.60183127,  6.21391719,  3.6069143 ,  3.61372985,  6.43895775,
        1.00005664,  6.95677456,  3.99578473,  5.39383825,  3.13368866,
        3.89330873,  8.0000052 ,  6.00006124,  7.92705939,  3.65626561,
        3.00000297,  3.44159412,  8.00002619,  8.99983741,  5.58263553,
        2.99995702,  4.90797934,  2.00000437,  4.9999249 ,  4.06802032,
        3.00004538,  4.13257519,  4.90265214,  1.99999519,  8.99999882,
        2.61602332,  6.421362  ,  7.31954807,  5.60780204,  4.31480469,
        1.99999803,  7.12388461,  5.85525171,  2.93790342,  5.78928622,
        5.84854148,  6.3180639 ,  4.81226428,  7.00006293,  6.02484481,
        3.87694476,  8.00003658,  2.96264326,  4.00001896,  2.13

In [52]:
z = np.reshape([int(round(i)) for i in slsqp_result], (9, -1))
z

array([[8, 6, 6, 1, 6, 3, 6, 4, 6],
       [4, 1, 3, 9, 4, 8, 4, 5, 6],
       [7, 6, 4, 6, 4, 4, 6, 1, 7],
       [4, 5, 3, 4, 8, 6, 8, 4, 3],
       [3, 8, 9, 6, 3, 5, 2, 5, 4],
       [3, 4, 5, 2, 9, 3, 6, 7, 6],
       [4, 2, 7, 6, 3, 6, 6, 6, 5],
       [7, 6, 4, 8, 3, 4, 2, 6, 5],
       [4, 6, 4, 3, 5, 7, 4, 7, 4]])

In [53]:
objective_func(slsqp_result)

1.1651079472791933e-07

# Genetic Algorithm

The fmin_slsqp result seemed so close, I was curious if running the GA on its output would tweak the last few values into place.  It turns out not to work.

In [62]:
def sudoku_fitness(a, b, attempt):
    return sum((np.dot(a, attempt) - b)**2)

def sudoku_mutation_swap(current, max_mutations=3):
    num_mutations = random.randint(1, max_mutations)
    baby = current[:]
    for i in range(num_mutations):
        index1 = random.randint(0,80)
        index2 = random.randint(0,80)
        baby[index1], baby[index2] = baby[index2], baby[index1]
    return baby

def sudoku_mutation_tweak(current, max_mutations=3):
    num_mutations = random.randint(1, max_mutations)
    baby = current[:]
    for i in range(max_mutations):
        index1 = random.randint(0,80)
        index2 = random.randint(0,80)
        tweak_amount = random.randint(1, 3)
        baby[index1] = ((baby[index1]+tweak_amount) % 9) + 1
        baby[index2] = ((baby[index2]+tweak_amount) % 9) + 1
    return baby

def sudoku_genetic_algorithm(fitness_func, mutation_func, initial_guess=None, max_iters=1000):
    iter_count = 0
    error_steps = []
    current = np.array([i for i in range(1,10) for j in range(1,10)])
    current_fitness = 100000000 # really big - assume the guess is really bad

    if initial_guess != None:
        current = initial_guess[:]
        current_fitness = fitness_func(current)
    
    for i in range(max_iters):
        baby = mutation_func(current)
        baby_fitness = fitness_func(baby)
        
        # If we found a perfect solution, just return it!
        if baby_fitness == 0:
            return (baby, 0)
        
        # Compare baby fitness to current
        elif baby_fitness < current_fitness:
            print("Doing switch: baby_fitness=%d, current_fitness=%d" % (baby_fitness, current_fitness))
            current = baby[:]
            current_fitness = baby_fitness
        
        iter_count += 1
        if iter_count % 10000 == 0:
            print("Error (steps=%d): %d" % (iter_count, current_fitness))
            error_steps.append(current_fitness)
            
    return (current, error_steps)

In [63]:
initial_g = np.array([int(round(i)) for i in slsqp_result])
fitness = lambda attempt: sudoku_fitness(p1_a, p1_b, attempt)
print("Initial fitness:", fitness(initial_g))
(ga_result, ga_error) = sudoku_genetic_algorithm(fitness, sudoku_mutation_tweak, initial_guess=initial_g, max_iters=50000)


Initial fitness: 11


/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:30: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Error (steps=10000): 11
Error (steps=20000): 11
Error (steps=30000): 11
Error (steps=40000): 11
Error (steps=50000): 11


In [64]:
np.reshape([int(round(i)) for i in ga_result], (9, -1))

array([[4, 5, 2, 2, 1, 4, 2, 4, 4],
       [7, 8, 3, 7, 7, 8, 3, 7, 9],
       [4, 9, 4, 1, 5, 5, 5, 7, 9],
       [7, 5, 1, 8, 4, 2, 8, 1, 1],
       [4, 5, 2, 5, 6, 4, 6, 3, 6],
       [4, 4, 8, 6, 1, 8, 5, 7, 8],
       [9, 3, 6, 7, 6, 5, 8, 4, 3],
       [6, 2, 8, 4, 3, 7, 9, 4, 5],
       [9, 1, 4, 7, 8, 4, 8, 9, 5]])

In [65]:
np.reshape([int(round(i)) for i in initial_g], (9, -1))

array([[4, 5, 2, 2, 1, 4, 2, 4, 4],
       [7, 8, 3, 7, 7, 8, 3, 7, 9],
       [4, 9, 4, 1, 5, 5, 5, 7, 9],
       [7, 5, 1, 8, 4, 2, 8, 1, 1],
       [4, 5, 2, 5, 6, 4, 6, 3, 6],
       [4, 4, 8, 6, 1, 8, 5, 7, 8],
       [9, 3, 6, 7, 6, 5, 8, 4, 3],
       [6, 2, 8, 4, 3, 7, 9, 4, 5],
       [9, 1, 4, 7, 8, 4, 8, 9, 5]])